In [3]:
# Part I

from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
# import enchant

In [4]:
def removeAbbreviations(statement):
    
    abbr = dict()

    abbr["ft"] = "foot"
    abbr["mm"] = "millimeter"
    abbr["cm"] = "centimeter"
    abbr["m"] = "meter"
    abbr["km"] = "kilometer"
    abbr["yd"] = "yard"
    abbr["mi"] = "mile"
    abbr["lb"] = "pound"
    abbr["oz"] = "ounce"
    abbr["mg"] = "milligram"
    abbr["g"] = "gram"
    abbr["kg"] = "kilogram"
    abbr["ml"] = "milliliter"
    abbr["l"] = "liter"
    abbr["mph"] = "miles per hour"
    abbr["kmph"] = "kilometers per hour"
    abbr["cu"] = "cubic"
    abbr["rpm"] = "revolutions per minute"
    abbr["&"] = "and"
    abbr["$"] = "dollar"
    abbr["°"] = "degree"
    abbr['"'] = "inch"
    
    statement_ = ""
    
    for word in statement.split():
        if word in abbr:
            statement_ = statement_ + " " + abbr[word]
        else:
            statement_ = statement_ + " " + word
            
    return statement_

In [5]:
def spelling(spellCheck, tokens1, tokens2):
    
    spell = SpellChecker()
    misspelled = spell.unknown(spellCheck)

    spellCheck_ = []
    for i in misspelled:
        spellCheck_.append(i)

    for i in range(len(spellCheck_)):
        spellCheck_[i] = spell.correction(spellCheck_[i])
    
    for word in spellCheck_:
        if not wordnet.synsets(word):
            spellCheck_.remove(word)
        
    for word in spellCheck_:
        flag1 = True
        flag2 = True
        for i in tokens1:
            if word == i:
                flag1 = False
                break
        for i in tokens2:
            if word == i:
                flag2 = False
                break
        if flag1 == True and flag2 == True:
            spellCheck_.remove(word)
    
    return spellCheck_

In [6]:
def checkSimilar(list1, list2):
    list1_copy = []
    for i in list1:
        list1_copy.append(i)
    list2_copy = []
    for i in list2:
        list2_copy.append(i)
    
    dict1 = dict()
    for i in list1:
        dict1[i] = 0
    for i in list2:
        dict1[i] = 0
    for i in list1:
        dict1[i] = 1 + dict1[i]
        
    dict2 = dict()
    for i in list1:
        dict2[i] = 0
    for i in list2:
        dict2[i] = 0
    for i in list2:
        dict2[i] = 1 + dict2[i]
        
    flag = True    
    for i in list1:
        if dict1[i] != dict2[i]:
            flag = False
            break
            
    return flag

In [7]:
def checkSimilarity(S1, S2):
    
    S1 = S1.lower()
    S2 = S2.lower()
    
    S1 = removeAbbreviations(S1)
    S2 = removeAbbreviations(S2)
    
    S1 = S1[1:len(S1)]
    S2 = S2[1:len(S2)]
    
    tokenizer = RegexpTokenizer(r'\w+')
    words1 = tokenizer.tokenize(S1)
    words2 = tokenizer.tokenize(S2)
        
    spellCheck = []
    for i in words1:
        flag = True
        for j in words2:
            if i == j:
                flag = False
                break
        if flag == True:
            spellCheck.append(i)
    for i in words2:
        flag = True
        for j in words1:
            if i == j:
                flag = False
                break
        if flag == True:
            spellCheck.append(i)
        
    corrected_spellings = spelling(spellCheck, words1, words2)
    
    for i in corrected_spellings:
        if i in spellCheck:
            spellCheck.remove(i)
    
    for i in range(len(corrected_spellings)):
        for j in range(len(words1)):
            if words1[j] == spellCheck[i]:
                words1[j] = corrected_spellings[i]
        for j in range(len(words2)):
            if words2[j] == spellCheck[i]:
                words2[j] = corrected_spellings[i]

    porter = PorterStemmer()

    lemmatize1 = []
    lemmatize2 = []

    for word in words1:
        lemmatize1.append(porter.stem(word))
    for word in words2:
        lemmatize2.append(porter.stem(word))
        
    for i in range(len(lemmatize1)):
        words1[i] = lemmatize1[i]
    for i in range(len(lemmatize2)):
        words2 = lemmatize2
    
    tokens1 = []
    for i in words1:
        tokens1.append(i)
    tokens2 = []
    for i in words2:
        tokens2.append(i)
    
    d = enchant.Dict("en_US")
    for word in words1:
        if d.check(word) == False:
            words1.remove(word)
    for word in words2:
        if d.check(word) == False:
            words2.remove(word)
            
    technical_terms1 = []
    for i in tokens1:
        check = False
        for j in words1:
            if i == j:
                check = True
                break
        if check == False:
            technical_terms1.append(i)

    technical_terms2 = []
    for i in tokens2:
        check = False
        for j in words2:
            if i == j:
                check = True
                break
        if check == False:
            technical_terms2.append(i)
            
    flag = False
    flag1 = True

    if len(technical_terms1) == len(technical_terms2) and len(technical_terms1) == 0:
        None
    elif len(technical_terms1) != len(technical_terms2):
        flag1 = False
    else:
        if checkSimilar(technical_terms1, technical_terms2):
            flag1 = True     

    if flag1 == True:
        from nltk.corpus import stopwords
        stop_words = set(stopwords.words('english'))
        stop_words1 = [w for w in words1 if not w in stop_words]
        stop_words2 = [w for w in words2 if not w in stop_words]

        flag = True
        if len(stop_words1) != len(stop_words2):
            flag = False
        else:
            flag = checkSimilar(stop_words1, stop_words2)

    return flag

# End of Part I